In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import torch
from models import nin_halfadd
import data
import util_fuc as util

def load_pretrained(filePath):
    model = nin_halfadd.Net()
    pretrained_model = torch.load(filePath)
    useState_dict = model.state_dict()
    preState_dict = pretrained_model['state_dict']
    useKeys = useState_dict.keys()
    preKeys = preState_dict.keys()
    j = 0
    print (len(useKeys))
    print (len(preKeys))
    for key in useKeys:
        if j == 50:
            j = 0
        if key.find('num_batches_tracked') == -1:
            useState_dict[key].data = preState_dict[preKeys[j]].data
            if key[-4:] != preKeys[j][-4:]:
                print (key , preKeys[j])
            j +=1
    model.load_state_dict(useState_dict)
    model.to(device)
    return model

In [5]:
testset = data.dataset(root='./data/', train=False)
testloader = torch.utils.data.DataLoader(testset,
                             batch_size=512, shuffle=False, num_workers=4)
device = torch.device('cuda:1' if torch.cuda.is_available() else "cpu")
model = load_pretrained('nin.best.pth.tar')
bin_op = util.BinOp(model)
bin_op.binarization()
model = model.eval()

64
50


In [6]:
correct = 0
with torch.no_grad():
    for image, target in testloader:
        image, target = image.to(device), target.to(device)

        output = model(image)
        #test_loss += criterion(output, target).data.item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum().cpu()
bin_op.restore()
acc = 100. * float(correct) / float(len(testloader.dataset))
print(acc)

86.28
